In [5]:
import os
from typing import TypedDict, Annotated
from langchain_google_genai import GoogleGenerativeAI
from langchain_core.messages import AnyMessage, HumanMessage, AIMessage
from langgraph.graph import StateGraph, END
from langgraph.graph.message import add_messages
from dotenv import load_dotenv
load_dotenv()
from langchain_community.tools.tavily_search import TavilySearchResults
google_api_key = os.getenv("GOOGLE_API_KEY")
tool=TavilySearchResults(max_results=2)

tools = [tool]

In [31]:
import os
from typing import TypedDict, Annotated, List
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_core.messages import BaseMessage, HumanMessage
from langgraph.graph import StateGraph, END
from langgraph.graph.message import add_messages
from langgraph.prebuilt import ToolNode
from langchain_community.tools.tavily_search import TavilySearchResults

search_tool = TavilySearchResults(max_results=2, name="tavily_search")
tools = [search_tool]


class AgentState(TypedDict):
    messages: Annotated[List[BaseMessage], add_messages]

model = ChatGoogleGenerativeAI(model="gemini-2.0-flash", temperature=0)

def call_model(state: AgentState):
    """The 'agent' node: calls the model to decide the next action."""
    print("---AGENT: CALLING MODEL---")
    messages = state['messages']

    response = model.invoke(messages, tools=tools)
    # The response, which may contain tool calls, is returned to be added to the state
    return {"messages": [response]}

tool_node = ToolNode(tools)

# --- 4. DEFINE ROUTER (CONDITIONAL EDGE) ---

def should_continue(state: AgentState) -> str:
    """
    Router logic: Inspects the last message to see if it contains tool calls.
    This function is made more robust by using `getattr` to safely access
    the .tool_calls attribute.
    """
    print("---ROUTER: CHECKING FOR NEXT STEP---")
    last_message = state['messages'][-1]
    
    # Safely check for tool_calls attribute
    if getattr(last_message, "tool_calls", None):
        print(">>> Decision: Tool call detected, routing to action.")
        return "action"
    else:
        print(">>> Decision: No tool call, routing to end.")
        return "end"

# --- 5. CONSTRUCT THE GRAPH ---
# The graph structure is identical to before

graph_builder = StateGraph(AgentState)

# Add the two nodes we defined: the agent and the tool executor
graph_builder.add_node("agent", call_model)
graph_builder.add_node("action", tool_node)

# The agent node is our entry point
graph_builder.set_entry_point("agent")

# Add the conditional edge for routing
graph_builder.add_conditional_edges(
    "agent",
    should_continue,
    {
        "action": "action",
        "end": END,
    },
)

# The action node loops back to the agent node
graph_builder.add_edge("action", "agent")

graph = graph_builder.compile()




### Without memory

In [32]:
events = graph.stream(
    {"messages": HumanMessage(content="what is the current price of nvidia stock?")},
)
for event in events:
    print("\n---\n")
    # The event gives us visibility into which node is running
    # and what its output is.
    print(event)

---AGENT: CALLING MODEL---
---ROUTER: CHECKING FOR NEXT STEP---
>>> Decision: Tool call detected, routing to action.

---

{'agent': {'messages': [AIMessage(content='', additional_kwargs={'function_call': {'name': 'tavily_search', 'arguments': '{"query": "nvidia stock price"}'}}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'model_name': 'gemini-2.0-flash', 'safety_ratings': []}, id='run--363017c9-f15e-4875-b778-772f995f0658-0', tool_calls=[{'name': 'tavily_search', 'args': {'query': 'nvidia stock price'}, 'id': '83eca96d-e5e1-4d25-9752-66969433c952', 'type': 'tool_call'}], usage_metadata={'input_tokens': 61, 'output_tokens': 8, 'total_tokens': 69, 'input_token_details': {'cache_read': 0}})]}}

---

{'action': {'messages': [ToolMessage(content='[{"title": "NVIDIA - 26 Year Stock Price History | NVDA - Macrotrends", "url": "https://www.macrotrends.net/stocks/charts/NVDA/nvidia/stock-price-history", "content": "Historical daily

### Adding Memeroy

In [ ]:
from langgraph.checkpoint.memory import MemorySaver

memory=MemorySaver()

graph=graph_builder.compile(checkpointer=memory)

config={
    {"configurable":{"threat_id":"user-123"}}
}



ModuleNotFoundError: No module named 'langgraph.checkpointer'